In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import scipy.signal
import radarkit
import matplotlib
import matplotlib.pyplot as plt
import time

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial', 'Helvetica']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rcParams['figure.figsize'] = (8.89, 5)   # Change the size of plots
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
sweep = radarkit.read('/Users/boonleng/Downloads/PX-20170220-050706-E2.4-Z.nc')

In [ ]:
z = sweep['moments'][0]['data']
p = sweep['moments'][4]['data']
r = sweep['moments'][5]['data']

In [ ]:
ia = np.arange(p.shape[0])
zc = np.copy(z)
zc[np.isnan(zc)] = -np.inf
z_lin = 10.0 ** (0.1 * zc)

In [ ]:
downSamplingRatio = int(sweep['gateSizeMeters'] / 30.0)

# Undo range correction and the ZCal in DSP to estimate the SNR
rr = 1.0e-3 * (np.arange(sweep['gateCount']) + 0.5) * sweep['gateSizeMeters']
snr = z - 20.0 * np.log10(rr) + 23

# Transition gate at 10.5 km at the given spacing
g = int((69.0e-6 * 3.0e8 * 0.5) / 30.0) + 5
g = int((g + downSamplingRatio - 1) / downSamplingRatio)

print(g)

z_off = 10.0 * np.log10(1.5 / 67)
snr[:, :g] = snr[:, :g] + z_off

# PhiDP calibration so that the transition is smooth and PhiDP starts ~ 0 deg
p[:, :g] = p[:, :g] + 40;
p[:, g:] = p[:, g:] + 44;

# A copy of PhiDP with NAN set to 0.0
pm = np.nan_to_num(np.copy(p))

In [ ]:
# hl = plt.plot(snr.T[g-20:g+20, :])
plt.figure()

plt.subplot(121)
_ = plt.plot(snr.T)
plt.title('Estimated SNR')

plt.subplot(122)
plt.imshow(snr)
plt.clim(-10, 60)
plt.colorbar(orientation='horizontal')
_ = plt.title('Estimated SNR')

### Fast Code

```python
s = time.time()
for _ in range(10):
    
    mask = np.logical_and(snr>0, r>0.85)

e = time.time()
print(e - s)
```

### Naive Code

```python
s = time.time()
for _ in range(10):
    
    mask = np.zeros(snr.shape, dtype=bool)
    for j in range(snr.shape[0]):
        for i in range(snr.shape[1]):
            if (snr[j, i] > 0 and r[j, i] > 0.85):
                mask[j, i] = True

e = time.time()
print(e - s)
```

In [ ]:
# Map NAN to some finite numbers for SNR and PhiDP
# snr[np.isnan(snr)] = -100.0
# r[np.isnan(r)] = 0.0

mask = np.logical_and(np.nan_to_num(np.copy(snr))>0, np.nan_to_num(r)>0.85)

In [ ]:
plt.figure()

plt.subplot(121)
plt.imshow(p, cmap=matplotlib.cm.hsv)
plt.clim(-45, 45)
plt.title('PhiDP (Degrees)')
plt.colorbar(orientation='horizontal')

plt.subplot(122)
plt.imshow(mask, cmap=matplotlib.cm.gray)
plt.clim(0, 1)
plt.title('Mask: SNR > 0 AND RhoHV > 0.85')
plt.colorbar(orientation='horizontal')

In [ ]:
# w = 10
# r0 = np.zeros(p.shape[0])
# for i in range(p.shape[0]):
#     for g in range(p.shape[1] - w):
#         kern = pm[i, g:g+w]
#         if np.std(kern) < 10.0 and np.mean(np.diff(kern)) > 0.1:
#             r0[i] = i
#             break

In [ ]:
# pp = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# ww = np.ones(2) / 2.0
# sp.signal.lfilter(ww, 1.0, pp)

In [ ]:
# Smoothing
w = 20
ww = np.ones(w) / w;
ps = sp.signal.lfilter(ww, 1.0, pm)

# Compute local variance
p_var = sp.signal.lfilter(ww, 1.0, pm ** 2) - ps ** 2
p_var[~mask] = 100.0

# Compute local slope
p_slope = sp.signal.lfilter(ww, 1.0, np.diff(p))

In [ ]:
plt.figure()

plt.subplot(121)
plt.imshow(p_var, cmap=matplotlib.cm.tab20c)
plt.clim(0.0, 100.0)
plt.title('Local Variance')
plt.colorbar(orientation='horizontal')

plt.subplot(122)
plt.imshow(p_slope, cmap=matplotlib.cm.RdYlBu)
plt.clim(-1.0, 1.0)
plt.title('Local Slope')
plt.colorbar(orientation='horizontal')

In [ ]:
good = np.logical_and(p_var < 20.0, mask)
good[:, 1:] = np.logical_and(np.abs(np.nan_to_num(p_slope, 0.0)) < 0.5, good[:, 1:])
# good[:, 1:] = np.logical_and(np.nan_to_num(p_slope, 0.0) > 0.01, good[:, 1:])

good_count = np.sum(good, axis=1)

g = 20;

# Data bounds
r0 = np.argmax(good[:, g+1:], axis=1) + g
rm = p.shape[1] - np.argmax(good[:, :g:-1], axis=1) - 1

# X-band
b = 1.02
alpha = np.arange(0.01, 0.051, 0.005)
alpha_count = len(alpha)

fourSixBDeltaR = 0.46 * b * 1.0e-3 * sweep['gateSizeMeters']

# Eq (15) for all (r; rm) so that ir[x] = 0.46 b int_x^rm (z ** b) dr
ir = fourSixBDeltaR * np.cumsum(z_lin[:, ::-1] ** b, axis=1)[:, ::-1]
ir0 = np.copy(ir)
# ir0 = [x[i] for x, i in zip(ir, r0)]
# ir0 = np.repeat(ir0, ir.shape[1]).reshape(ps.shape)

paths = []
ah = np.zeros((*ps.shape, len(alpha)))
edge = np.zeros(good.shape, dtype=bool)
deltaPhi = np.zeros(ps.shape[0])
for i, s, e, c in zip(ia, r0, rm, good_count):
    if c > 50:
        # Only use the path index if the length > 50 cells
        edge[i, s:e] = True;
        paths.append((i, s, e))
        deltaPhi[i] = ps[i, e] - ps[i, s]
        ir[i, :] = ir[i, s]
        
z_lin[~edge] = 0.0

In [ ]:
# The common term in size of (360, 1, alpha_count)
tenPowerSomethingMinusOne = 10.0 ** (0.1 * b * np.outer(deltaPhi, alpha).reshape((ps.shape[0], 1, -1)))

In [ ]:
z_lin_big = np.repeat(z_lin, alpha_count).reshape((*ps.shape, alpha_count))
ir_big = np.repeat(ir, alpha_count).reshape((*ps.shape, alpha_count))
ir0_big = np.repeat(ir0, alpha_count).reshape((*ps.shape, alpha_count))

num = z_lin_big ** b * tenPowerSomethingMinusOne
den = (ir0_big + tenPowerSomethingMinusOne * ir_big)
den[den == 0] = 1.0
ah = num / den

In [ ]:
alpha_big = np.outer(np.ones(ps.shape), alpha).reshape((*ps.shape, alpha_count))
p_con = 2.0 * np.cumsum(ah, axis=1) / alpha_big * 1.0e-3 * sweep['gateSizeMeters']

In [ ]:
ps = np.nan_to_num(ps)
p_con = np.nan_to_num(p_con)
err = np.sum(np.abs(np.repeat(ps, alpha_count).reshape((*ps.shape, alpha_count)) - p_con), axis=(0,1))

alpha_idx = np.argmin(err)

In [ ]:
ps[~edge] = np.nan

In [ ]:
plt.figure()

plt.subplot(121)
plt.imshow(ps, cmap=matplotlib.cm.hsv)
plt.clim(-45, 45)
plt.title('Smoothed PhiDP')
plt.colorbar(orientation='horizontal')

plt.subplot(122)
plt.imshow(edge, cmap=matplotlib.cm.gray)
plt.clim(0, 1)
plt.title('Good')
plt.colorbar(orientation='horizontal')

In [ ]:
plt.figure()

d = p_con[:, :, alpha_idx]
d[~edge] = np.nan

plt.subplot(121)
plt.imshow(d, cmap=matplotlib.cm.hsv)
plt.clim(-45, 45)
plt.title('Constructed PhiDP (alpha = {:.4f})'.format(alpha[alpha_idx]))
plt.colorbar(orientation='horizontal')

plt.subplot(122)
plt.imshow(edge, cmap=matplotlib.cm.gray)
plt.clim(0, 1)
plt.title('Good')
plt.colorbar(orientation='horizontal')